In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
import os
import sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from utils.BiographyVectorStore import retrieve_vector_store
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
# Initialize the vector store
vector_store_dir = os.path.join(os.path.dirname(os.getcwd()), 'data/vector_store')
vectorstore = retrieve_vector_store(vector_store_dir, HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': False}
))

# Test vectorstore is successfully retrieved
print(f"Type of vectorstore: {type(vectorstore)}")
print(f"Number of documents in the vector store: {vectorstore._collection.count()}")
print(vectorstore.similarity_search("chimpanzees"))

/home/mparoca/anaconda3/envs/pd_simulation/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/mparoca/anaconda3/envs/pd_simulation/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Type of vectorstore: <class 'langchain_chroma.vectorstores.Chroma'>
Number of documents in the vector store: 10
[Document(metadata={'id': '52cfe727-fa49-4559-a4bc-84aa88d3dbd9', 'name': 'Jane Goodall'}, page_content='Dame Jane Morris Goodall  (; born Valerie Jane Morris-Goodall; 3 April 1934), formerly Baroness Jane van Lawick-Goodall, is an English zoologist, primatologist and anthropologist. She is considered the world\'s foremost expert on chimpanzees, after 60 years\' studying the social and family interactions of wild chimpanzees. Goodall first went to Gombe Stream National Park in Tanzania to observe its chimpanzees in 1960.\nShe is the founder of the Jane Goodall Institute and the Roots & Shoots programme, and she has worked extensively on conservation and animal welfare issues. As of 2022, she is on the board of the Nonhuman Rights Project. In April 2002, she was named a United Nations Messenger of Peace. Goodall is an honorary member of the World Future Council.\n\n\n== Early 

In [ ]:
# Define a Pydantic model for the structured output
# class GameDecision(BaseModel):
#     decision: str = Field(description="The decision made by the agent, either 'C' or 'D'")
#     reasoning: str = Field(description="The reasoning behind the decision")


In [39]:
class GameAgent:
    def __init__(self, agent_name: str, vectorstore, system_message: str):
        self.agent_name = agent_name
        self.vectorstore = vectorstore
        self.retriever = vectorstore.as_retriever()
        #model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)
        #self.model = model.with_structured_output(GameDecision)
        #self.model = ChatOpenAI(model="gpt-4o", temperature=0)
        self.model = ChatOpenAI(model="gpt-4o", temperature=0.5)
        self.memory = MemorySaver()
        self.tools = [TavilySearchResults(max_results=3)] 
        self.agent_executor = create_react_agent(self.model, self.tools, checkpointer=self.memory)
        # Initialize chat history with system message
        self.chat_history = [SystemMessage(content=system_message)]
        self.config = {"configurable": {"thread_id": "abc123"}}

    def add_prompt_message(self, content: str):
        self.chat_history.append(HumanMessage(content=content))

    # def get_agent_response(self, prompt: str, opponent_name: str):
    #     # The system message already contains background info
    #     self.add_prompt_message(prompt)
    #     final_message = ""
    #     print(f"Prompt to {self.agent_name}:")
    #     print(prompt)
    #     for chunk in self.agent_executor.stream({"messages": self.chat_history}, self.config):
    #         final_message = chunk
    #     response = final_message['agent']['messages'][0].content.strip().upper()
    #     # Remove last message from chat history
    #     self.chat_history.pop()
    #     return response
    
    def get_agent_response(self, prompt: str, opponent_name: str):
        # The system message already contains background info
        self.add_prompt_message(prompt)
        final_message = ""
        reasoning = ""  # Variable to store reasoning
        print(f"Prompt to {self.agent_name}:")
        print(prompt)
        for chunk in self.agent_executor.stream({"messages": self.chat_history}, self.config):
            reasoning_chunk = chunk['agent']['messages'][0].content.strip()
            reasoning += reasoning_chunk  # Accumulate reasoning chunks
        response = reasoning[0].upper()  # Assuming the first character is the decision
        # Remove last message from chat history
        self.chat_history.pop()
        return response, reasoning
    # def get_agent_response(self, prompt: str, opponent_name: str):
    #     self.add_prompt_message(prompt)
    #     print(f"Prompt to {self.agent_name}:")
    #     print(prompt)
        
    #     # Use the agent executor to handle the interaction
    #     for chunk in self.agent_executor.stream({"messages": self.chat_history}, self.config):
    #         final_message = chunk
        
    #     # Assuming the final_message contains the structured output
    #     decision = final_message.decision
    #     reasoning = final_message.reasoning
        
    #     # Remove last message from chat history
    #     self.chat_history.pop()
    #     return decision, reasoning

    # def retrieve_information(self, name: str) -> str:
    #     # Use the vector store to retrieve information about the person
    #     try:
    #         results = self.vectorstore.similarity_search(name, k=1)
    #     except Exception as e:
    #         print(f"Error during retrieval: {e}")
    #         results = []
    #     if results:
    #         full_text = results[0].page_content
    #     # Limit the text to, say, 500 characters
    #         summary = full_text[:8000]
    #         return summary
    #     else:
    #         return f"No background information available for {name}."
    
    def retrieve_information(self, name: str) -> str:
        # Use the vector store to retrieve information about the person
        try:
            retriever = self.vectorstore.as_retriever(
                search_kwargs={"k": 1, "filter": {"name": name}}, 
            )
            # Perform the search using the retriever
            results = retriever.invoke("")
        except Exception as e:
            print(f"Error during retrieval: {e}")
            results = []
        if results:
            full_text = results[0].page_content
            # Limit the text to 8000 characters
            summary = full_text[:10000]
            return summary
        else:
            return f"No background information available for {name}."

# def retrieve_information(self, name: str) -> str:
#     # Use the retriever to retrieve information about the person using metadata
#     try:
#         # Assuming the retriever supports metadata filtering
#         results = self.retriever.retrieve(
#             query=name,
#             filters={"name": name},  # Filter by the 'name' metadata field
#             k=1  # Retrieve the single most relevant document
#         )
#     except Exception as e:
#         print(f"Error during retrieval: {e}")
#         results = []

#     if results:
#         # Since each person is in one document, take the first result
#         full_text = results[0].page_content
#         # Limit the text to, say, 8000 characters
#         summary = full_text[:8000]
#         return summary
#     else:
#         return f"No background information available for {name}."

In [6]:
# test retrieve information
def retrieve_information(name):
    # Use the vector store to retrieve information about the person
    try:
        retriever = vectorstore.as_retriever(
            search_kwargs={"k": 50, "filter": {"name": name}}, 
        )
        # Perform the search using the retriever
        results = retriever.invoke("")
    except Exception as e:
        print(f"Error during retrieval: {e}")
        results = []
    if results:
        full_text = results[0].page_content
        # Limit the text to 8000 characters
        summary = full_text[:8000]
        return full_text
    else:
        return f"No background information available for {name}."

# Test function
print(retrieve_information(name="Elon Musk"))




Number of requested results 50 is greater than number of elements in index 10, updating n_results = 10


Elon Reeve Musk  (; born June 28, 1971) is a businessman and investor known for his key roles in the space company SpaceX and the automotive company Tesla, Inc. Other involvements include ownership of X Corp., the company that operates the social media platform X (formerly known as Twitter), and his role in the founding of the Boring Company, xAI, Neuralink, and OpenAI. He is one of the wealthiest individuals in the world; as of August 2024 Forbes estimates his net worth to be US$247 billion.
Musk was born in Pretoria, South Africa, to Maye (née Haldeman), a model, and Errol Musk, a businessman and engineer. Musk briefly attended the University of Pretoria before immigrating to Canada at the age of 18, acquiring citizenship through his Canadian-born mother. Two years later he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics and physics. He moved to California in 1995 to attend

In [49]:
def get_system_prompt():
    sys_prompt = (
        "Embody the character of{agent_name} during an interaction with {opponent_name}."
        "You have the following background information.\n\n"
        "Your Background:\n{agent_info}\n\n"
        #"Opponent's Background:\n{opponent_info}\n\n" # Uncomment this if you want agents to consider their opponent's background
        "Your task is to fully embody the thoughts, motivations, and personality of {agent_name}, staying true to their known character traits and values. \n\n"
        "Game Instructions:\n"
        "In each round, you have two choices: Cooperate (C) or Defect (D). \n"
        #"You have the option to **cooperate** with {opponent_name} for mutual benefit or to **act independently** for potential personal gain.\n"
        "Your response should reflect your unique inner motivations, values, history,and goals.\n"
        "Make your decision based on how you, {agent_name} would act in this situation and on your natural tendencies and history, not how others might perceive"    
        )
    return sys_prompt

def get_task_prompt(your_past_moves, your_score, opponent_past_moves, opponent_score):
    task_prompt = (
        "Previous Interactions:\n\n"
        f"Your past moves: {your_past_moves}\n"
        #f"Your Score: {your_score}\n" #Uncomment if you want agents to consider their score
        f"Opponent's past moves: {opponent_past_moves}\n"
        #f"Opponent's Score: {opponent_score}\n\n"
        "Current Situation:\n\n"
        "For this round, you must choose either to Cooperate (C) or Defect (D).\n"
        "Do not use any of your tools or external resources to make this decision.\n\n"
        "Rely soley on your biography to inform your decision\n{agent_info}\n\n"
        "Output Instructions: Your response should include your decision as one character, either 'C' or 'D', "
        "followed by a brief explanation of your reasoning behind the decision." 
        "Do not include any additional text or explanation beyond this."
        "**Response Format:**\n"
        "C or D - [Your reasoning as the character you are embodying and the past history of your interactions with the opponent].\n"
        "For example:\n"
        "C - I have longed worked with chimpazees in the wild and animals display cooperative strategies to ensure their survival.\n"
        "D - War is a necessary tool to ensure peace and stability. I have led many wars. When the enemy retaliates, we must strike first to ensure our survival.\n"

    )

    return task_prompt


In [8]:
class PrisonersDilemmaGame:
    def __init__(self, num_rounds, agent1_name, agent2_name, vectorstore):
        self.vectorstore = vectorstore
        self.agent1_name = agent1_name
        self.agent2_name = agent2_name
        self.agent1 = GameAgent(agent_name=agent1_name, vectorstore=vectorstore, system_message="")
        self.agent2 = GameAgent(agent_name=agent2_name, vectorstore=vectorstore, system_message="")
        self.num_rounds = num_rounds
        self.history1 = []
        self.history2 = []
        self.score1 = 0
        self.score2 = 0
        self.scores_round1 = []
        self.scores_round2 = []
        self.reasoning1 = []
        self.reasoning2 = []

        # Retrieve background information
        agent1_info = self.agent1.retrieve_information(agent1_name)
        agent2_info = self.agent2.retrieve_information(agent2_name)

        # Create system messages
        system_message1 = get_system_prompt().format(
            agent_name=self.agent1_name,
            opponent_name=self.agent2_name,
            agent_info=agent1_info,
            opponent_info=agent2_info
        )

        system_message2 = get_system_prompt().format(
            agent_name=self.agent2_name,
            opponent_name=self.agent1_name,
            agent_info=agent2_info,
            opponent_info=agent1_info
        )

        # Set system messages
        self.agent1.chat_history = [SystemMessage(content=system_message1)]
        self.agent2.chat_history = [SystemMessage(content=system_message2)]

    def play(self):
        for round_number in range(1, self.num_rounds + 1):
            print(f"Round {round_number}")

            # Generate prompts for both agents
            agent1_prompt = get_task_prompt(self.history1, self.score1, self.history2, self.score2)
            agent2_prompt = get_task_prompt(self.history2, self.score2, self.history1, self.score1)

            # Get decisions from both agents
            #decision1 = self.agent1.get_agent_response(agent1_prompt, self.agent2_name)
            #decision2 = self.agent2.get_agent_response(agent2_prompt, self.agent1_name)
            
            # Get reasoning from both agents
            decision1, reasoning1 = self.agent1.get_agent_response(agent1_prompt, self.agent2_name)
            decision2, reasoning2 = self.agent2.get_agent_response(agent2_prompt, self.agent1_name)
            
            # Store reasoning
            self.reasoning1.append(reasoning1)
            self.reasoning2.append(reasoning2)

            # Update histories
            self.history1.append(decision1)
            self.history2.append(decision2)

            # Update scores
            self.update_scores(decision1, decision2)

            # Store scores for each round
            self.scores_round1.append(self.score1)
            self.scores_round2.append(self.score2)

            # Print the results of the round
            print(f"{self.agent1_name} Decision: {decision1}, {self.agent2_name} Decision: {decision2}")
            print(f"Scores -> {self.agent1_name}: {self.score1}, {self.agent2_name}: {self.score2}\n")
            print(f"{self.agent1_name} Reasoning: {reasoning1}")
            print(f"{self.agent2_name} Reasoning: {reasoning2}")

    def update_scores(self, decision1, decision2):
        payoff_matrix = {
            ('C', 'C'): (3, 3),
            ('C', 'D'): (0, 5),
            ('D', 'C'): (5, 0),
            ('D', 'D'): (1, 1)
        }
        result = payoff_matrix.get((decision1, decision2), (0, 0))
        self.score1 += result[0]
        self.score2 += result[1]

    def get_scores(self):
        """Returns the accumulated scores per round for both agents."""
        return self.scores_round1, self.scores_round2


In [50]:
# Assuming vectorstore is already initialized
agent1_name = "Pope Francis"
agent2_name = "Benjamin Netanyahu"

game = PrisonersDilemmaGame(num_rounds=5, agent1_name=agent1_name, agent2_name=agent2_name, vectorstore=vectorstore)
game.play()

final_scores = game.get_scores()
print("Final Scores:")
print(f"{agent1_name}: {final_scores[0][-1]}")
print(f"{agent2_name}: {final_scores[1][-1]}")


Round 1
Prompt to Pope Francis:
Previous Interactions:

Your past moves: []
Opponent's past moves: []
Current Situation:

For this round, you must choose either to Cooperate (C) or Defect (D).
Do not use any of your tools or external resources to make this decision.

Rely soley on your biography to inform your decision
{agent_info}

Output Instructions: Your response should include your decision as one character, either 'C' or 'D', followed by a brief explanation of your reasoning behind the decision.Do not include any additional text or explanation beyond this.**Response Format:**
C or D - [Your reasoning as the character you are embodying and the past history of your interactions with the opponent].
For example:
C - I have longed worked with chimpazees in the wild and animals display cooperative strategies to ensure their survival.
D - War is a necessary tool to ensure peace and stability. I have led many wars. When the enemy retaliates, we must strike first to ensure our survival.

